In [19]:
from sklearn import *
import os
import pickle
import pandas as pd
import swifter
import numpy as np
from sklearn.linear_model import LogisticRegression
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import re #regular expressions
from bs4 import BeautifulSoup
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

[nltk_data] Downloading package stopwords to /home/becode/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
os.getcwd()

'/home/becode/dr-robin/Enron/robinchaos'

In [5]:
file = 'data/maarten/all_mails.csv'
df = pd.read_csv(file)

In [7]:
model = pickle.load(open('spam__ham.pickle','rb'))


LogisticRegression(C=10)

In [173]:
tf_transformer = pickle.load(open('spam__ham__tf.pickle','rb'))
count_vec = pickle.load(open('spam__ham__vec_bow.pickle','rb'))

In [177]:
print(count_vec)
print(tf_transformer)

CountVectorizer()


In [245]:
#Load model, and tf_transformer and count_vec from pickle files
model = pickle.load(open('spam__ham.pickle','rb'))
tf_transformer = pickle.load(open('spam__ham__tf.pickle','rb'))
count_vec = pickle.load(open('spam__ham__vec_bow.pickle','rb'))


def spampredict(x):
    language='english'
    minWordSize=3
    # remove non-letters
    text_alpha_chars = re.sub("[^a-zA-Z']", " ", str(x))

    # convert to lower-case
    text_lower = text_alpha_chars.lower()

    # remove stop words
    stops = set(stopwords.words(language))
    text_no_stop_words = ' '
    #print(text_no_stop_words)
    for w in text_lower.split():
        if w not in stops:
            text_no_stop_words = text_no_stop_words + w + ' '
            #print(text_no_stop_words)
    # do stemming
    text_stemmer = ' '
    stemmer = SnowballStemmer(language)
    for w in text_no_stop_words.split():
        text_stemmer = text_stemmer + stemmer.stem(w) + ' '
        #print(text_stemmer)
    
    # remove short words
    processedtext = ' '
    for w in text_stemmer.split():
        
        if len(w) >= minWordSize:
            processedtext + w + ' '
            
            processedtext = [processedtext]

            X_bag_of_words = count_vec.transform(processedtext)
            
            X_tf = tf_transformer.transform(X_bag_of_words)
            
            spampredict = model.predict(X_tf)
            
            
            return spampredict

In [246]:
#Run spam filter on allmails.csv
df['spamfilter'] = df['Content'].swifter.apply(lambda x : spampredict(x))